In [1]:
import os
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\kidney_disease\\research'

In [2]:
os.chdir('../')
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\kidney_disease'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Jeevan880/kidney_disease.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Jeevan880"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d44b672f466e4ad3749288c4810b53ae4108f713"

In [4]:
import tensorflow as tf

In [5]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [7]:
from kidney_disease.constant import *
from kidney_disease.utils.common import read_yaml,save_json,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self)->EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts\data_ingestion\kidney-ct-scan-image",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/Jeevan880/kidney_disease.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
    
    def _valid_generator(self):
        try:
            import PIL
            print("Pillow version:", PIL.__version__)
        except ImportError:
            print("Pillow is not installed.")
            
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.3
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)
    def evaluate_model(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=self.model.evaluate(self.valid_generator)
        self.save_score()
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [11]:
import tqdm

In [12]:
try:
    config=ConfigurationManager()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluate_model()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-16 23:23:21,170: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-16 23:23:21,185: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-16 23:23:21,193: INFO: common: created directory at: artifacts]


Pillow version: 11.1.0
Found 93 images belonging to 2 classes.
6/6 [==============================] - 44s 7s/step - loss: 1.9626 - accuracy: 0.6559
[2025-04-16 23:24:07,611: INFO: common: json file saved at: scores.json]


2025/04/16 23:24:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-04-16 23:24:15,743: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Tester\AppData\Local\Temp\tmpk00mz3rz\model\data\model\assets
[2025-04-16 23:24:18,540: INFO: builder_impl: Assets written to: C:\Users\Tester\AppData\Local\Temp\tmpk00mz3rz\model\data\model\assets]


2025/04/16 23:25:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/04/16 23:25:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


🏃 View run inquisitive-koi-420 at: https://dagshub.com/Jeevan880/kidney_disease.mlflow/#/experiments/0/runs/7c3b8a4890734a0abf31b16ed6496868
🧪 View experiment at: https://dagshub.com/Jeevan880/kidney_disease.mlflow/#/experiments/0
